<a href="https://colab.research.google.com/github/ramprakashyadav/OCR/blob/main/SpaCY(NER)_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U spacy -q

     |████████████████████████████████| 6.0 MB 15.6 MB/s 
     |████████████████████████████████| 181 kB 71.3 MB/s 
     |████████████████████████████████| 628 kB 77.6 MB/s 
     |████████████████████████████████| 451 kB 79.5 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 10.1 MB 62.0 MB/s 


In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [4]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [5]:
TRAIN_DATA

{'annotations': [['SI Description of Goods HSN/SAC | Quantity Rate per | Disc. % Amount',
   {'entities': [[34, 51, 'QUANTITY'], [62, 68, 'AMOUNT']]}],
  ['No', {'entities': []}],
  ['1 | Laryngeal Mask Airway Normal 1.0 (LMA110) 90192010) Spes| 365.30| pcs 1,776.50',
   {'entities': [[4, 36, 'VENDOR NAME'],
     [56, 61, 'QUANTITY'],
     [70, 82, 'AMOUNT']]}],
  ['Batch : 20200513 5S pes', {'entities': []}],
  ['2 | Laryngeal Mask Airway Reinforced 2.0 (LMA220) }90192010/ 20 pes; 355.30)pcs 7,106.00',
   {'entities': [[4, 40, 'VENDOR NAME'],
     [61, 67, 'QUANTITY'],
     [69, 75, 'AMOUNT'],
     [80, 88, 'AMOUNT']]}],
  ['Batch ; 20200513 20 pes', {'entities': []}],
  ['3 | Endortacheal Tube CUFFED 6.5 (ET0065) 90189023} 20 pes| 50.00| pcs 1,000.00',
   {'entities': [[4, 32, 'VENDOR NAME'],
     [52, 59, 'QUANTITY'],
     [60, 66, 'AMOUNT'],
     [71, 79, 'AMOUNT']]}],
  ['Batch ; 20200427 20 pes |', {'entities': []}],
  ['4 | Endortacheal Tube CUFFED 7.0 (ET0070) |90189029| 20 pes

In [6]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          print("Skipping entity")
        else:
          ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")        


100%|██████████| 134/134 [00:00<00:00, 2119.26it/s]

Skipping entity


In [7]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-01-27 12:08:34,324] [INFO] Set up nlp object from config
[2022-01-27 12:08:34,336] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-01-27 12:08:34,341] [INFO] Created vocabulary
[2022-01-27 12:08:34,342] [INFO] Finished initializing nlp object
[2022-01-27 12:08:34,580] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     62.89    0.00    0.00    0.00    0.00
 29     200        311.09   3279.01   95.15   96.08   94.23    0.95
 64     400        443.68    774.00   98.08   98.08   98.08    0.98
107     600        494.96    731.32   98.08   98

In [9]:
nlp_ner = spacy.load("/content/model-best")

In [10]:
doc = nlp_ner ('''TUOREN MEDICAL DEVICE INDIA PVT LTD 17th Floor, OG-B-17-012,.DG-B-17-014
Emaar Digital Greens,Golf Course Extension Road
Sector-61, Gurugram, PIN No.122011

DL No.:-HR-6600-894 Dt 18/07/18
GSTIN/UIN: O6AAGCT3779A1ZD_

State Name : Haryana, Code : 06

GIN: U33309DL2017FTG321882

E-Mail : salesadmin@tuoren.in

 

Buyer

Medical Equipment Co.

30-A, Maqbara Road, Hazratganj
Lucknow, Pin No. 226001

GSTIN/UIN : O9AAMFM9696C1ZM
State Name : Uttar Pradesh, Code : 09


invoice No. Dated
229/20-21 __ 31-Dec-2020

 

 

Delivery Note Mode/Terms of Payment

 

 

45 Days
Suppliers Ref. ‘Other Reference(s)
635
Buyer's Order No. Dated
635 31-Dec-2020

Despatch Document No. | Delivery Note Date

Despatched through _|Destination

 

Terms of Delivery

 


Tax Amount (in words) : INR Seventeen Thousand Two Hundred Thirty Eight and Thirty paise Only

 
   
   

ical Device india P
company's PAN : AAGCT3778A For Tuoren Mee

Jjeciaration f for TUOREN MEDICAL DE!
Ve declare that this invoice shows the actual price of the |
joods described and that all particulars are true and correct.

 

LTD]

yoga

 

‘This Is a Computer Generated Invoice


SI Description of Goods HSN/SAC | Quantity Rate per | Disc. % Amount
No
1 | Laryngeal Mask Airway Normal 1.0 (LMA110) 90192010) Spes| 365.30| pcs 1,776.50
Batch : 20200513 5S pes
2 | Laryngeal Mask Airway Reinforced 2.0 (LMA220) }90192010/ 20 pes; 355.30)pcs 7,106.00
Batch ; 20200513 20 pes
3 | Endortacheal Tube CUFFED 6.5 (ET0065) 90189023} 20 pes| 50.00| pcs 1,000.00
Batch ; 20200427 20 pes |
4 | Endortacheal Tube CUFFED 7.0 (ET0070) |90189029| 20 pes) 50.00! pcs 1,000.00
Batch : 20200427 20 pes,
5 | Endortacheal Tube CUFFED 7.5 (ET0075) 90189029) 20 pes| 50.00 pes 1,000.00
Batch : 20200627 20 pes.
6 | Endortacheal Tube CUFFED 8.0 (ET0080) 90189028 20 pes! 50.00] pcs 1,000.00
Batch : 20191214 20 pes
7 | Endortacheal Tube CUFFED 8.5 (ET0085) 90189023} 20 pes| —_50.00| pcs 1,000.00
Batch : 20191214 20 pcs
8 |Endotracheal Tube Reinforced CUFFED 7.0 (RET0070) [90189028] 90 pcs| 307.00] pcs 27,630.00
| Batch : 20190923 90 pcs) |
9 |Endotracheal Tube Reinforced CUFFED 7.5 (RET0075) [90189029100 pes| 307.00] pcs 30,700.00
Batch : 20190919 | 100 pes |
10| Endotracheal Tube Reinforced CUFFED 8.0 (RET0080) [90189029|100 pes| 307.00| pcs 30,700.00
Batch : 20190923 | 100 pes
11 Endotracheal Tube Reinforced CUFFED 6.5(RET0065) 90189029! 60 pcs| 307.00] pcs 18,420.00
Batch : 20190320 60 pcs.
12| Silicone Anaesthetia Mask 3 (SMB003) 90189041| 5 pes} 550.00| pcs: 2,750.00
Batch : 20200413 5 pes
13| Silicone Anaesthetia Mask 4 (SMB004) 90189041} 5S pes} 550.00) pcs 2,750.00
Batch : 20200413 Specs
14 Silicone Anaesthetia Mask 5 (SMB005) 90189041/ § pes| 550.00/pcs| 2,750.00
Batch ; 20200413 5 pcs: |
15| Endotracheal Tube BB-7.0 (BET0070) |90189029] 10 pes| 350.00/pcs 3,500.00
Batch > 20191014 10 pes
16| Endotracheal Tube BB-7.5 (BETO075) 90189029) 10 pcs 350.00} pes: 3,500.00
Batch ; 20191028 10 pos
17|Endobronchial Tube DLL Fr35 (DENL035) 90189029| 5 pes| 1,414.00| pcs, 7,070.00
Batch : 20190830 3 pes.
Batch ; 20191218 2 pcs) |
7,43,652.50
IGST 12% | 12|% 17,238.30
Total 515 pes! © 4,60,890.80
‘Amount Chargeable (in words) - . E80.
INR One Lakh Sixty Thousand Eight Hundred Ninety and Eighty paise Only
HSNISACG Taxable | Integrated Tax | Total
Be = Value [Rate | Amount _|Tax Amount
‘30792010 8,882.50/ 12%| 1,065.90| 1,065.90
90189029 1,26,520.00| 12%| 15,182.40) 15,182.40
|90189041 | _ z 12%)''')




In [11]:
spacy.displacy.render(doc, style="ent", jupyter=True)